In [19]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

Mounted at /content/drive
Mounted at /content/gdrive


In [20]:
from numpy import save, load
from pandas import read_csv
from tensorflow.keras import layers
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense,Activation
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset5/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset5/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset5/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset5/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


In [22]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)

In [23]:
# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

In [24]:
embed_dim = x_train.shape[-1]   # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = x_train.shape[-1]  # Hidden layer size in feed forward network inside transformer

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm

In [26]:
# Define per-fold score containers 
acc_per_fold = []
loss_per_fold = []
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
input_shape = x_train.shape[1:]
input = Input(shape=x_train.shape[1:])
for train, test in kfold.split(inputs, targets):

  model = keras.Sequential()
  model.add(layers.Input(shape= x_train.shape[1:]))
  model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
  model.add(layers.GlobalAveragePooling1D())
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(ff_dim, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  history = model.fit(inputs[train], targets[train], epochs=200, verbose=True, batch_size=128,callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)])
    # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

model.summary()



------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/200
464/464 [==============================] - 9s 16ms/step - loss: 0.3277 - accuracy: 0.8649
Epoch 2/200
464/464 [==============================] - 7s 16ms/step - loss: 0.2668 - accuracy: 0.8952
Epoch 3/200
464/464 [==============================] - 7s 15ms/step - loss: 0.2433 - accuracy: 0.9048
Epoch 4/200
464/464 [==============================] - 7s 16ms/step - loss: 0.2287 - accuracy: 0.9098
Epoch 5/200
464/464 [==============================] - 7s 16ms/step - loss: 0.2165 - accuracy: 0.9146
Epoch 6/200
464/464 [==============================] - 7s 16ms/step - loss: 0.2088 - accuracy: 0.9177
Epoch 7/200
464/464 [==============================] - 7s 16ms/step - loss: 0.2126 - accuracy: 0.9157
Epoch 8/200
464/464 [==============================] - 7s 16ms/step - loss: 0.1937 - accuracy: 0.9232
Epoch 9/200
464/464 [==============================] - 7s 16ms/step - loss: 0.1907 - ac

In [27]:
# Average scores 
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.21203435957431793 - Accuracy: 93.45876574516296%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.25529178977012634 - Accuracy: 93.8431441783905%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.25565269589424133 - Accuracy: 93.76180171966553%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.21858999133110046 - Accuracy: 93.64041090011597%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.2219267189502716 - Accuracy: 93.56622695922852%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 93.6540699005127 (+- 0.13667618017427807)
> Loss: 0.23269911110401154
------------------------